In [313]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [314]:
from IPython.display import display, HTML
display(HTML(data="""
<style>
    div#notebook-container    { width: 90%; }
    div#menubar-container     { width: 65%; }
    div#maintoolbar-container { width: 90%; }
</style>
"""))
pd.set_option('display.max_columns', 100)

In [315]:
df_rated_book_per_user =pd.read_csv('C://Users//rupadhyay//Desktop//ML//Analytics Summit Bibliotheca E Book subscription prediction//Data//df_highest_rated_filtered.csv',encoding='latin1')

In [316]:
df_master_test =pd.read_csv('C://Users//rupadhyay//Desktop//ML//Analytics Summit Bibliotheca E Book subscription prediction//Data//BOOKSMASTERTEST.csv',encoding='latin1')

In [317]:
df_master_test.drop('GENRE.1', axis=1, inplace=True)

In [74]:
df_master_test.isnull().sum()

BookID                0
GENRE                 0
BOOKTITLE             0
BOOKNAME              0
SERIES              393
AUTHOR                0
USERRATINGS           0
OTHERPRINTEDINFO      0
DETAILS               0
SUMMARY               0
AUTHORDESC            0
COVERPAGE             0
Popularity            0
dtype: int64

In [318]:
df_master_test.drop('SERIES', axis=1, inplace=True)

In [319]:
df_master_test['PAGES'] = df_master_test['OTHERPRINTEDINFO'].str.extract('(?P<PAGES>\d* (?=pages))', expand=True)

In [320]:
df_master_test['PUBLISH_YEAR'] = df_master_test['OTHERPRINTEDINFO'].str.extract('(?P<PUBLISH_YEAR>\d* (?=by))', expand=True)

In [321]:
df_master_test['PAGES'].isnull().sum()

13

In [322]:
df_master_test['PAGES'].fillna(0, inplace=True)
df_master_test['PAGES'] = df_master_test['PAGES'].astype(str).astype(int)
df_master_test['PAGES'] = df_master_test['PAGES'].replace(0,df_master_test['PAGES'].mean())

In [323]:
df_master_test['PUBLISH_YEAR'].isnull().sum()

8

In [324]:
df_master_test['PUBLISH_YEAR'].fillna(0, inplace=True)

In [325]:
df_master_test.PUBLISH_YEAR[df_master_test.PUBLISH_YEAR == ' '] = 0
df_master_test['PUBLISH_YEAR'] = df_master_test['PUBLISH_YEAR'].astype(str).astype(int)

c:\users\rupadhyay\python 3.6\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [83]:
df_master_test.head(5)

,BookID,GENRE,BOOKTITLE,BOOKNAME,AUTHOR,USERRATINGS,OTHERPRINTEDINFO,DETAILS,SUMMARY,AUTHORDESC,COVERPAGE,Popularity,PAGES,PUBLISH_YEAR
0,109615.0,Fiction,The Girls' Guide to Hunting and Fishing by Mel...,The Girls' Guide to Hunting and Fishing,Melissa Bank,3.31,"['Paperback, 288 pages', 'Published March 29th...","[('Original Title', ""The Girls' Guide to Hunti...","Generous-hearted and wickedly insightful, The ...","""Melissa Bank (born in 1961 in Philadelphia) i...",https://images.gr-assets.com/books/1309210476l...,1883.961984,288.0,2005
1,148159.0,Nonfiction,How Full Is Your Bucket?: Positive Strategies ...,How Full Is Your Bucket?: Positive Strategies ...,Tom Rath,3.86,"[""Hardcover, Educator's Edition, 224 pages"", '...","[('Original Title', 'How Full Is Your Bucket? ...","Based on a simple metaphor, the book features ...","ARE YOU FULLY CHARGED?, the latest national b...",https://images.gr-assets.com/books/1386923784l...,88.060588,224.0,2007
2,117280.0,Fiction,Eine gefangene Liebe by Anita Shreve,Eine gefangene Liebe,Anita Shreve,3.29,"['Paperback, 252 pages', 'Published September ...","[('Original Title', 'Where or When'), ('ISBN',...",Charles Callahan is reading the Sunday paper w...,"Anita Shreve was an American writer, chiefly k...",https://images.gr-assets.com/books/1172425279l...,99.257499,252.0,1999
3,164937.0,Classics,The Resurrection by Leo Tolstoy,The Resurrection,Leo Tolstoy,4.12,"['Audio CD, 17 pages', 'Published March 1st 20...","[('Original Title', 'ÐÐ¾ÑÐºÑÐµÑÐµÐ½Ð¸Ðµ'),...",Leo Tolstoy stands tall among the great Russia...,Lev Nikolayevich Tolstoy (Russian: ÐÐµÐ² ÐÐ¸...,https://images.gr-assets.com/books/1347520060l...,212.280962,17.0,2005
4,172865.0,Nonfiction,Thrive: 30 Inspirational Rags-to-Riches Storie...,Thrive: 30 Inspirational Rags-to-Riches Stories,Jason Navallo,3.86,"['Kindle Edition, 100 pages', 'Published April...","[('ASIN', 'B00WNHB59I'), ('Edition Language', ...",Thrive: 30 Inspirational Rags-to-Riches Storie...,Jason Navallo is the author of five books (Ame...,https://images.gr-assets.com/books/1430867773l...,4.039204,100.0,2015


In [326]:
df_test_books = df_master_test[['BookID','PAGES','PUBLISH_YEAR','GENRE','BOOKNAME','AUTHOR','SUMMARY']].copy()

In [327]:
df_test_books.head(5)

,BookID,PAGES,PUBLISH_YEAR,GENRE,BOOKNAME,AUTHOR,SUMMARY
0,109615.0,288.0,2005,Fiction,The Girls' Guide to Hunting and Fishing,Melissa Bank,"Generous-hearted and wickedly insightful, The ..."
1,148159.0,224.0,2007,Nonfiction,How Full Is Your Bucket?: Positive Strategies ...,Tom Rath,"Based on a simple metaphor, the book features ..."
2,117280.0,252.0,1999,Fiction,Eine gefangene Liebe,Anita Shreve,Charles Callahan is reading the Sunday paper w...
3,164937.0,17.0,2005,Classics,The Resurrection,Leo Tolstoy,Leo Tolstoy stands tall among the great Russia...
4,172865.0,100.0,2015,Nonfiction,Thrive: 30 Inspirational Rags-to-Riches Stories,Jason Navallo,Thrive: 30 Inspirational Rags-to-Riches Storie...


In [328]:
df_test_books['AUTHOR'] = df_test_books['AUTHOR'].str.replace(' ', '')

In [329]:
df_test_books.head(5)

,BookID,PAGES,PUBLISH_YEAR,GENRE,BOOKNAME,AUTHOR,SUMMARY
0,109615.0,288.0,2005,Fiction,The Girls' Guide to Hunting and Fishing,MelissaBank,"Generous-hearted and wickedly insightful, The ..."
1,148159.0,224.0,2007,Nonfiction,How Full Is Your Bucket?: Positive Strategies ...,TomRath,"Based on a simple metaphor, the book features ..."
2,117280.0,252.0,1999,Fiction,Eine gefangene Liebe,AnitaShreve,Charles Callahan is reading the Sunday paper w...
3,164937.0,17.0,2005,Classics,The Resurrection,LeoTolstoy,Leo Tolstoy stands tall among the great Russia...
4,172865.0,100.0,2015,Nonfiction,Thrive: 30 Inspirational Rags-to-Riches Stories,JasonNavallo,Thrive: 30 Inspirational Rags-to-Riches Storie...


In [330]:
df_test_books['AUTHOR'] = df_test_books['AUTHOR'].str.lower()

In [331]:
# Creating new column for summary keywords
df_test_books['KEY_WORDS'] = ""

for index, row in df_test_books.iterrows():
    plot = row['SUMMARY']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)
    

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_ranked_phrases()
    
    # assigning the key words to the new column
    #row['Key_words'] = key_words_dict_scores
    df_test_books.at[index,'KEY_WORDS'] = list(key_words_dict_scores)



In [332]:
df_test_books.head(5)

,BookID,PAGES,PUBLISH_YEAR,GENRE,BOOKNAME,AUTHOR,SUMMARY,KEY_WORDS
0,109615.0,288.0,2005,Fiction,The Girls' Guide to Hunting and Fishing,melissabank,"Generous-hearted and wickedly insightful, The ...","[young woman coming, unforgettable comic touch..."
1,148159.0,224.0,2007,Nonfiction,How Full Is Your Bucket?: Positive Strategies ...,tomrath,"Based on a simple metaphor, the book features ...","[bonus section â  â  instructional guide, ..."
2,117280.0,252.0,1999,Fiction,Eine gefangene Liebe,anitashreve,Charles Callahan is reading the Sunday paper w...,"[oddly familiar photo catches, correspondence ..."
3,164937.0,17.0,2005,Classics,The Resurrection,leotolstoy,Leo Tolstoy stands tall among the great Russia...,"[two together many years later, beautiful youn..."
4,172865.0,100.0,2015,Nonfiction,Thrive: 30 Inspirational Rags-to-Riches Stories,jasonnavallo,Thrive: 30 Inspirational Rags-to-Riches Storie...,"[including jim carrey, become wealthy entrepre..."


In [333]:
# dropping the SUMMARY column
df_test_books.drop(columns = ['SUMMARY'], inplace = True)

In [334]:
df_test_books.head(5)

,BookID,PAGES,PUBLISH_YEAR,GENRE,BOOKNAME,AUTHOR,KEY_WORDS
0,109615.0,288.0,2005,Fiction,The Girls' Guide to Hunting and Fishing,melissabank,"[young woman coming, unforgettable comic touch..."
1,148159.0,224.0,2007,Nonfiction,How Full Is Your Bucket?: Positive Strategies ...,tomrath,"[bonus section â  â  instructional guide, ..."
2,117280.0,252.0,1999,Fiction,Eine gefangene Liebe,anitashreve,"[oddly familiar photo catches, correspondence ..."
3,164937.0,17.0,2005,Classics,The Resurrection,leotolstoy,"[two together many years later, beautiful youn..."
4,172865.0,100.0,2015,Nonfiction,Thrive: 30 Inspirational Rags-to-Riches Stories,jasonnavallo,"[including jim carrey, become wealthy entrepre..."


In [335]:
df_test_books.dtypes

BookID          float64
PAGES           float64
PUBLISH_YEAR      int32
GENRE            object
BOOKNAME         object
AUTHOR           object
KEY_WORDS        object
dtype: object

In [336]:
df_test_books['BookID'] = df_test_books['BookID'].astype(int)
df_test_books['PAGES'] = df_test_books['PAGES'].astype(int)

In [337]:
df_indexed_test_books= df_test_books.set_index('BookID')


In [338]:
df_indexed_test_books.head(5)

,PAGES,PUBLISH_YEAR,GENRE,BOOKNAME,AUTHOR,KEY_WORDS
BookID,,,,,,
109615,288,2005,Fiction,The Girls' Guide to Hunting and Fishing,melissabank,"[young woman coming, unforgettable comic touch..."
148159,224,2007,Nonfiction,How Full Is Your Bucket?: Positive Strategies ...,tomrath,"[bonus section â  â  instructional guide, ..."
117280,252,1999,Fiction,Eine gefangene Liebe,anitashreve,"[oddly familiar photo catches, correspondence ..."
164937,17,2005,Classics,The Resurrection,leotolstoy,"[two together many years later, beautiful youn..."
172865,100,2015,Nonfiction,Thrive: 30 Inspirational Rags-to-Riches Stories,jasonnavallo,"[including jim carrey, become wealthy entrepre..."


In [339]:
df_indexed_test_books.dtypes

PAGES            int32
PUBLISH_YEAR     int32
GENRE           object
BOOKNAME        object
AUTHOR          object
KEY_WORDS       object
dtype: object

In [340]:
df_indexed_test_books['PUBLISH_YEAR'] = df_indexed_test_books['PUBLISH_YEAR'].astype(str)
df_indexed_test_books['PAGES'] = df_indexed_test_books['PAGES'].astype(str)

In [341]:
df_indexed_test_books['bag_of_words'] = ''
columns = df_indexed_test_books.columns
for index, row in df_indexed_test_books.iterrows():
    words = ''
    for col in columns:
        if col == 'KEY_WORDS':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '

    df_indexed_test_books.at[index,'bag_of_words'] = words
    


In [342]:
df_indexed_test_books.head(5)

,PAGES,PUBLISH_YEAR,GENRE,BOOKNAME,AUTHOR,KEY_WORDS,bag_of_words
BookID,,,,,,,
109615,288,2005,Fiction,The Girls' Guide to Hunting and Fishing,melissabank,"[young woman coming, unforgettable comic touch...",288 2005 Fiction The Girls' Guide to Hunting a...
148159,224,2007,Nonfiction,How Full Is Your Bucket?: Positive Strategies ...,tomrath,"[bonus section â  â  instructional guide, ...",224 2007 Nonfiction How Full Is Your Bucket?: ...
117280,252,1999,Fiction,Eine gefangene Liebe,anitashreve,"[oddly familiar photo catches, correspondence ...",252 1999 Fiction Eine gefangene Liebe anitashr...
164937,17,2005,Classics,The Resurrection,leotolstoy,"[two together many years later, beautiful youn...",17 2005 Classics The Resurrection leotolstoy t...
172865,100,2015,Nonfiction,Thrive: 30 Inspirational Rags-to-Riches Stories,jasonnavallo,"[including jim carrey, become wealthy entrepre...",100 2015 Nonfiction Thrive: 30 Inspirational R...


In [343]:
df_indexed_test_books.drop(columns = [col for col in df_indexed_test_books.columns if col!= 'bag_of_words'], inplace = True)

In [344]:
df_indexed_test_books.head(5)

,bag_of_words
BookID,
109615,288 2005 Fiction The Girls' Guide to Hunting a...
148159,224 2007 Nonfiction How Full Is Your Bucket?: ...
117280,252 1999 Fiction Eine gefangene Liebe anitashr...
164937,17 2005 Classics The Resurrection leotolstoy t...
172865,100 2015 Nonfiction Thrive: 30 Inspirational R...


In [230]:
#df_non_index_test = df_indexed_test_books.copy()

In [231]:
#df_non_index_test['BookID']= df_non_index_test.index

In [233]:
#df_indexed_test_books = df_indexed_test_books[['BookID','bag_of_words']]

## TEST DATA

In [234]:
#df_indexed_test_books.head(5)

,BookID,bag_of_words
BookID,,
109615,109615,288 2005 Fiction The Girls' Guide to Hunting a...
148159,148159,224 2007 Nonfiction How Full Is Your Bucket?: ...
117280,117280,252 1999 Fiction Eine gefangene Liebe anitashr...
164937,164937,17 2005 Classics The Resurrection leotolstoy t...
172865,172865,100 2015 Nonfiction Thrive: 30 Inspirational R...


In [166]:
df_indexed_test_books.to_csv('../Data/Latest/df_indexed_test_books.csv')

### converting rated df for cosine similarity

In [167]:
df_rated_book_per_user.head(5)

,Unnamed: 0,UserID,BookID,GENDER,STATE,GENRE,BOOKNAME,AUTHOR,Age_Category,PAGES,PUBLISH_YEAR,SUMMARY
0,163854,646637,112318,FEMALE,NaN,Nonfiction,A 2nd Helping of Chicken Soup for the Soul: 10...,Jack Canfield,45-54,446.0,1996,"The runaway bestseller ""Chicken Soup for the S..."
1,4621,621033,119228,MALE,Delaware,Classics,Pride and Prejudice,Jane Austen,35-44,279.0,2000,"Since its immediate success in 1813, Pride and..."
2,348037,667563,141194,MALE,Texas,Nonfiction,The South Beach Diet Supercharged: Faster Weig...,Arthur Agatston,45-54,337.0,2008,"Five years ago, with the publication of The So..."
3,3922,656318,119228,MALE,NaN,Classics,Pride and Prejudice,Jane Austen,55-64,279.0,2000,"Since its immediate success in 1813, Pride and..."
4,105383,662455,134452,FEMALE,Iowa,Classics,Pride and Prejudice,Jane Austen,25-34,279.0,2000,"Since its immediate success in 1813, Pride and..."


In [345]:
df_master_to_rate = df_rated_book_per_user[['UserID','BookID','PAGES','PUBLISH_YEAR','GENRE','BOOKNAME','AUTHOR','SUMMARY']].copy()

In [346]:
df_master_to_rate['AUTHOR'] = df_master_to_rate['AUTHOR'].str.replace(' ', '')
df_master_to_rate['AUTHOR'] = df_master_to_rate['AUTHOR'].str.lower()

In [482]:
df_master_to_rate.head(5)

,UserID,BookID,bag_of_words
0,646637,112318,446 1996 Nonfiction A 2nd Helping of Chicken S...
1,621033,119228,279 2000 Classics Pride and Prejudice janeaust...
2,667563,141194,337 2008 Nonfiction The South Beach Diet Super...
3,656318,119228,279 2000 Classics Pride and Prejudice janeaust...
4,662455,134452,279 2000 Classics Pride and Prejudice janeaust...


In [348]:
# Creating new column for summary keywords
df_master_to_rate['KEY_WORDS'] = ""

for index, row in df_master_to_rate.iterrows():
    plot = row['SUMMARY']
    
    # instantiating Rake, by default is uses english stopwords from NLTK
    # and discard all puntuation characters
    r = Rake()

    # extracting the words by passing the text
    r.extract_keywords_from_text(plot)
    

    # getting the dictionary whith key words and their scores
    key_words_dict_scores = r.get_ranked_phrases()
    
    # assigning the key words to the new column
    #row['Key_words'] = key_words_dict_scores
    df_master_to_rate.at[index,'KEY_WORDS'] = list(key_words_dict_scores)



In [349]:
df_master_to_rate.head(5)

,UserID,BookID,PAGES,PUBLISH_YEAR,GENRE,BOOKNAME,AUTHOR,SUMMARY,KEY_WORDS
0,646637,112318,446.0,1996,Nonfiction,A 2nd Helping of Chicken Soup for the Soul: 10...,jackcanfield,"The runaway bestseller ""Chicken Soup for the S...","[nation still hungering, hansen went back, wor..."
1,621033,119228,279.0,2000,Classics,Pride and Prejudice,janeausten,"Since its immediate success in 1813, Pride and...","[radiant wit sparkles, jane austen called, jan..."
2,667563,141194,337.0,2008,Nonfiction,The South Beach Diet Supercharged: Faster Weig...,arthuragatston,"Five years ago, with the publication of The So...","[find inspiring success stories, effective wei..."
3,656318,119228,279.0,2000,Classics,Pride and Prejudice,janeausten,"Since its immediate success in 1813, Pride and...","[radiant wit sparkles, jane austen called, jan..."
4,662455,134452,279.0,2000,Classics,Pride and Prejudice,janeausten,"Since its immediate success in 1813, Pride and...","[radiant wit sparkles, jane austen called, jan..."


In [350]:
# dropping the SUMMARY column
df_master_to_rate.drop(columns = ['SUMMARY'], inplace = True)

In [351]:
df_master_to_rate.dtypes

UserID            int64
BookID            int64
PAGES           float64
PUBLISH_YEAR      int64
GENRE            object
BOOKNAME         object
AUTHOR           object
KEY_WORDS        object
dtype: object

In [352]:
df_master_to_rate['BookID'] = df_master_to_rate['BookID'].astype(int)
df_master_to_rate['PAGES'] = df_master_to_rate['PAGES'].astype(int)
df_master_to_rate['PUBLISH_YEAR'] = df_master_to_rate['PUBLISH_YEAR'].astype(int)
df_master_to_rate['UserID'] = df_master_to_rate['UserID'].astype(int)

In [353]:
df_master_to_rate['BookID'] = df_master_to_rate['BookID'].astype(str)
df_master_to_rate['PAGES'] = df_master_to_rate['PAGES'].astype(str)
df_master_to_rate['PUBLISH_YEAR'] = df_master_to_rate['PUBLISH_YEAR'].astype(str)
df_master_to_rate['UserID'] = df_master_to_rate['UserID'].astype(str)

In [354]:
df_master_to_rate.head(5)

,UserID,BookID,PAGES,PUBLISH_YEAR,GENRE,BOOKNAME,AUTHOR,KEY_WORDS
0,646637,112318,446,1996,Nonfiction,A 2nd Helping of Chicken Soup for the Soul: 10...,jackcanfield,"[nation still hungering, hansen went back, wor..."
1,621033,119228,279,2000,Classics,Pride and Prejudice,janeausten,"[radiant wit sparkles, jane austen called, jan..."
2,667563,141194,337,2008,Nonfiction,The South Beach Diet Supercharged: Faster Weig...,arthuragatston,"[find inspiring success stories, effective wei..."
3,656318,119228,279,2000,Classics,Pride and Prejudice,janeausten,"[radiant wit sparkles, jane austen called, jan..."
4,662455,134452,279,2000,Classics,Pride and Prejudice,janeausten,"[radiant wit sparkles, jane austen called, jan..."


In [355]:
df_master_to_rate['bag_of_words'] = ''
columns = ['PAGES','PUBLISH_YEAR','GENRE','BOOKNAME','AUTHOR','KEY_WORDS']
for index, row in df_master_to_rate.iterrows():
    words = ''
    for col in columns:
        if col == 'KEY_WORDS':
            words = words + ' '.join(row[col])+ ' '
        else:
            words = words + row[col]+ ' '

    df_master_to_rate.at[index,'bag_of_words'] = words
    


In [356]:
df_master_to_rate.head(5)

,UserID,BookID,PAGES,PUBLISH_YEAR,GENRE,BOOKNAME,AUTHOR,KEY_WORDS,bag_of_words
0,646637,112318,446,1996,Nonfiction,A 2nd Helping of Chicken Soup for the Soul: 10...,jackcanfield,"[nation still hungering, hansen went back, wor...",446 1996 Nonfiction A 2nd Helping of Chicken S...
1,621033,119228,279,2000,Classics,Pride and Prejudice,janeausten,"[radiant wit sparkles, jane austen called, jan...",279 2000 Classics Pride and Prejudice janeaust...
2,667563,141194,337,2008,Nonfiction,The South Beach Diet Supercharged: Faster Weig...,arthuragatston,"[find inspiring success stories, effective wei...",337 2008 Nonfiction The South Beach Diet Super...
3,656318,119228,279,2000,Classics,Pride and Prejudice,janeausten,"[radiant wit sparkles, jane austen called, jan...",279 2000 Classics Pride and Prejudice janeaust...
4,662455,134452,279,2000,Classics,Pride and Prejudice,janeausten,"[radiant wit sparkles, jane austen called, jan...",279 2000 Classics Pride and Prejudice janeaust...


In [357]:
df_master_to_rate.drop(columns = [col for col in ['PAGES','PUBLISH_YEAR','GENRE','BOOKNAME','AUTHOR','KEY_WORDS'] ], inplace = True)

In [368]:
df_master_to_rate.head(5)

,UserID,BookID,bag_of_words
0,646637,112318,446 1996 Nonfiction A 2nd Helping of Chicken S...
1,621033,119228,279 2000 Classics Pride and Prejudice janeaust...
2,667563,141194,337 2008 Nonfiction The South Beach Diet Super...
3,656318,119228,279 2000 Classics Pride and Prejudice janeaust...
4,662455,134452,279 2000 Classics Pride and Prejudice janeaust...


In [185]:
df_master_to_rate.to_csv('../Data/Latest/df_master_to_rate.csv')

### FINDING COSINE SIMILARITY

In [187]:
#df_indexed_test_books

In [436]:
df_master_to_rate_cosine = df_master_to_rate[['BookID','bag_of_words']].copy()


In [437]:
df_train_books = df_master_to_rate_cosine.copy()

In [438]:
df_train_books= df_train_books.set_index('BookID')

In [439]:
df_train_books.head(5)

,bag_of_words
BookID,
112318,446 1996 Nonfiction A 2nd Helping of Chicken S...
119228,279 2000 Classics Pride and Prejudice janeaust...
141194,337 2008 Nonfiction The South Beach Diet Super...
119228,279 2000 Classics Pride and Prejudice janeaust...
134452,279 2000 Classics Pride and Prejudice janeaust...


In [272]:
#df_master_to_rate_cosine= df_master_to_rate_cosine.set_index('BookID')

In [236]:
#train_indices = pd.Series(df_master_to_rate_cosine.index)

In [237]:
#test_indices = pd.Series(df_indexed_test_books.index)

In [238]:
#df_appended = df_indexed_test_books.append(df_master_to_rate_cosine)

In [239]:
#df_appended.head(5)

,BookID,bag_of_words
109615,109615,288 2005 Fiction The Girls' Guide to Hunting a...
148159,148159,224 2007 Nonfiction How Full Is Your Bucket?: ...
117280,117280,252 1999 Fiction Eine gefangene Liebe anitashr...
164937,164937,17 2005 Classics The Resurrection leotolstoy t...
172865,172865,100 2015 Nonfiction Thrive: 30 Inspirational R...


In [211]:
def diff(first, second):
        second = set(second)
        return [item for item in first if item not in second]

to_rate_indices = diff(indices, master_indices) 

In [375]:
### SETTING UP MULTIPLE INDEX FOR GETTING RECOMMENDATIONS
#df_train_books.set_index(['UserID', 'BookID'], inplace=True)

In [445]:
df_train_books.head(5)

,bag_of_words
BookID,
112318,446 1996 Nonfiction A 2nd Helping of Chicken S...
119228,279 2000 Classics Pride and Prejudice janeaust...
141194,337 2008 Nonfiction The South Beach Diet Super...
119228,279 2000 Classics Pride and Prejudice janeaust...
134452,279 2000 Classics Pride and Prejudice janeaust...


In [440]:
df_indexed_test_books.head(5)

,bag_of_words
BookID,
109615,288 2005 Fiction The Girls' Guide to Hunting a...
148159,224 2007 Nonfiction How Full Is Your Bucket?: ...
117280,252 1999 Fiction Eine gefangene Liebe anitashr...
164937,17 2005 Classics The Resurrection leotolstoy t...
172865,100 2015 Nonfiction Thrive: 30 Inspirational R...


In [441]:
df_master_to_rate_cosine.index = df_master_to_rate_cosine.index.astype(int)
df_indexed_test_books.index = df_indexed_test_books.index.astype(int)
df_train_books.index = df_train_books.index.astype(int)

# FINAL TRY

In [446]:
recommended_books = [[]]
for index, row in df_train_books.iterrows():
    #val = row['bag_of_words']
    d = { 'BookID': [index] , 'bag_of_words': [row['bag_of_words']] }
    temp_df = pd.DataFrame.from_dict(d)
    temp_df.set_index('BookID', inplace=True)
    #temp_df = pd.DataFrame(row['bag_of_words'], columns = ['bag_of_words']) 
    #temp_df = df_train_books[[df_train_books['bag_of_words']== row['bag_of_words']]]
    #temp_df = temp_df[df_train_books.index]
    #temp_df = pd.DataFrame(df_train_books.loc[index], columns = ['bag_of_words']) 
    df_appended = df_indexed_test_books.append(temp_df)
    count = CountVectorizer()
    count_matrix = count.fit_transform(df_appended['bag_of_words'])
    cosine_sim = cosine_similarity(count_matrix, count_matrix)
    indices = pd.Series(df_appended.index)
    indices = indices.drop_duplicates()
    
    
    title = index
    # gettin the index of the book that matches the title
    idx = indices[indices == title].index[0]

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar books
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching books
    for i in top_10_indexes:
        recommended_books.append([title, (list(df_appended.index)[i])])
    
    
    df_appended = df_appended.sort_values(by=[('BookID')])
    df_appended.truncate(before=-1, after=-1)
    temp_df = df_appended.sort_values(by=[('BookID')])
    temp_df.truncate(before=-1, after=-1)
        

In [444]:
recommended_books

[[]]

In [447]:
recommended_dataset = pd.DataFrame(recommended_books, columns=['BookID','Recommendations'])

In [448]:
recommended_dataset.dropna(inplace=True)

In [449]:

recommended_dataset['BookID'] = recommended_dataset['BookID'].astype(int)
recommended_dataset['Recommendations'] = recommended_dataset['Recommendations'].astype(int)
recommended_dataset['BookID'] = recommended_dataset['BookID'].astype(str)
recommended_dataset['Recommendations'] = recommended_dataset['Recommendations'].astype(str)

In [450]:
recom_df = (recommended_dataset.groupby('BookID')['Recommendations']
       .apply(lambda x: ','.join(set(x.dropna())))
       .reset_index())


## TOP 10 TEST books matching with books in TRAIN set

In [464]:
recom_df.head(5)

,BookID,Recommendations
0,100075,"179004,123373,123203,173264,134563,141325,1334..."
1,100080,"155103,127616,172503,129475,125368,147894,1649..."
2,100086,"169243,156728,123203,173264,134563,141325,1299..."
3,100104,"126752,131233,104657,137665,107611,126661,1617..."
4,100106,"126587,166252,165392,173377,167058,144850,1583..."


In [473]:
import csv
recom_df.to_csv('../Data/Output/recom_df.csv',quoting=csv.QUOTE_ALL)

In [463]:
recom_df.dtypes

BookID             object
Recommendations    object
dtype: object

In [467]:
r_df = recom_df.copy()

In [475]:
r_df[['1','2','3','4','5','6','7','8','9','10']] = r_df['Recommendations'].str.split(',',expand=True)

In [481]:
r_df.head(5)

,BookID,1,2,3,4,5,6,7,8,9,10
0,100075,179004,123373,123203,173264,134563,141325,133443,130390,173551,136824
1,100080,155103,127616,172503,129475,125368,147894,164931,143914,104018,109538
2,100086,169243,156728,123203,173264,134563,141325,129944,173551,139128,148942
3,100104,126752,131233,104657,137665,107611,126661,161780,115795,134525,131196
4,100106,126587,166252,165392,173377,167058,144850,158378,152621,143596,121184


In [477]:
r_df.to_csv('../Data/Output/r_df.csv')

In [480]:
r_df.drop(['Recommendations'],axis=1, inplace=True)

## Predicting books for users in Master train file

In [483]:
train_predictions = df_master_to_rate[['UserID','BookID']].copy()

In [486]:
train_predictions.head()

,UserID,BookID
0,646637,112318
1,621033,119228
2,667563,141194
3,656318,119228
4,662455,134452


In [485]:
train_predictions.shape

(12628, 2)

In [487]:
train_predictions_mergerd = pd.merge(train_predictions, r_df, on='BookID', how='inner')

In [488]:
train_predictions_mergerd.shape

(12628, 12)

In [489]:
train_predictions_mergerd.head()

,UserID,BookID,1,2,3,4,5,6,7,8,9,10
0,646637,112318,152447,165998,163740,113159,156638,102128,124775,122295,139128,162878
1,657735,112318,152447,165998,163740,113159,156638,102128,124775,122295,139128,162878
2,607478,112318,152447,165998,163740,113159,156638,102128,124775,122295,139128,162878
3,621033,119228,100727,168509,127652,127616,177441,129218,142477,113594,165976,165523
4,656318,119228,100727,168509,127652,127616,177441,129218,142477,113594,165976,165523


In [490]:
train_predictions_mergerd.to_csv('../Data/Output/train_predictions_mergerd.csv')

## Making prediction on missing data

In [526]:
df_missing =pd.read_csv('C://Users//rupadhyay//Desktop//ML//Analytics Summit Bibliotheca E Book subscription prediction//Data//New//MISSING.csv',encoding='latin1')

In [527]:
df_missing.head()

,UserID
0,600072
1,600079
2,600099
3,600152
4,600158


In [493]:
df_usr_mstr =pd.read_csv('C://Users//rupadhyay//Desktop//ML//Analytics Summit Bibliotheca E Book subscription prediction//Data//USERMASTER.csv',encoding='latin1')

In [528]:
df_missing_data = pd.merge(df_missing, df_usr_mstr, on='UserID', how='inner')

In [529]:
df_missing_data.shape

(2372, 6)

In [530]:
df_missing_data.head()

,UserID,SIGNUPDATE,DELETED,AGEGROUP,GENDER,STATE
0,600072,2018-08-20 03:00:30,NaN,49.0,FEMALE,NaN
1,600079,2018-09-21 20:33:30,NaN,53.0,FEMALE,NaN
2,600099,2017-04-16 15:31:23,NaN,27.0,MALE,Georgia
3,600152,2018-09-07 00:35:51,NaN,37.0,MALE,Ohio
4,600158,2018-01-29 21:57:59,NaN,33.0,MALE,North Carolina


In [531]:
df_missing_data = df_missing_data [['UserID','AGEGROUP','GENDER']]

In [532]:
df_missing_data.head()

,UserID,AGEGROUP,GENDER
0,600072,49.0,FEMALE
1,600079,53.0,FEMALE
2,600099,27.0,MALE
3,600152,37.0,MALE
4,600158,33.0,MALE


In [533]:
def categorize_age(age):
    if age < 18: return '0-17'
    elif age < 25: return '18-24'
    elif age < 35: return '25-34'
    elif age < 45: return '35-44'
    elif age < 55: return '45-54'
    elif age < 65: return '55-64'
    else: return '65+'
 

In [534]:
df_missing_data['AGEGROUP'] = df_missing_data.AGEGROUP.apply(lambda x:categorize_age(x))

In [535]:
from sklearn.preprocessing import Imputer, LabelEncoder
df_missing_data['GENDER'] = LabelEncoder().fit_transform(df_missing_data['GENDER'])
df_missing_data['AGEGROUP'] = LabelEncoder().fit_transform(df_missing_data['AGEGROUP'])

In [505]:
df_book_age_highest_rating =pd.read_csv('C://Users//rupadhyay//Desktop//ML//Analytics Summit Bibliotheca E Book subscription prediction//Data//Output//df_book_age_highest.csv',encoding='latin1')

In [536]:
df_missing_merge = pd.merge(df_missing_data, df_book_age_highest_rating, on=('AGEGROUP','GENDER'), how='inner')

In [537]:
df_missing_merge.head()

,UserID,AGEGROUP,GENDER,Unnamed: 0,BookID
0,600072,4,0,163854,112318
1,600079,4,0,163854,112318
2,600162,4,0,163854,112318
3,600501,4,0,163854,112318
4,602030,4,0,163854,112318


In [538]:
df_missing_merge.shape

(2372, 5)

In [539]:
df_missing_merge.drop(['AGEGROUP','GENDER','Unnamed: 0'], axis=1, inplace=True)

In [540]:
df_missing_merge.dtypes

UserID    int64
BookID    int64
dtype: object

In [541]:
df_missing_merge['BookID'] = df_missing_merge['BookID'].astype(str)
df_missing_merge['UserID'] = df_missing_merge['UserID'].astype(str)

In [542]:
missing_predictions_final = pd.merge(df_missing_merge, r_df, on='BookID', how='inner')

In [543]:
missing_predictions_final.head()

,UserID,BookID,1,2,3,4,5,6,7,8,9,10
0,600072,112318,152447,165998,163740,113159,156638,102128,124775,122295,139128,162878
1,600079,112318,152447,165998,163740,113159,156638,102128,124775,122295,139128,162878
2,600162,112318,152447,165998,163740,113159,156638,102128,124775,122295,139128,162878
3,600501,112318,152447,165998,163740,113159,156638,102128,124775,122295,139128,162878
4,602030,112318,152447,165998,163740,113159,156638,102128,124775,122295,139128,162878


In [544]:
missing_predictions_final.shape

(2372, 12)

In [545]:
missing_predictions_final.to_csv('../Data/Output/missing_predictions_final.csv')  #Predictions for missing members

In [546]:
predictions_final_append = train_predictions_mergerd.append(missing_predictions_final)

In [547]:
predictions_final_append.shape

(15000, 12)

In [548]:
predictions_final_append.head()

,UserID,BookID,1,2,3,4,5,6,7,8,9,10
0,646637,112318,152447,165998,163740,113159,156638,102128,124775,122295,139128,162878
1,657735,112318,152447,165998,163740,113159,156638,102128,124775,122295,139128,162878
2,607478,112318,152447,165998,163740,113159,156638,102128,124775,122295,139128,162878
3,621033,119228,100727,168509,127652,127616,177441,129218,142477,113594,165976,165523
4,656318,119228,100727,168509,127652,127616,177441,129218,142477,113594,165976,165523


In [549]:
predictions_final_append.to_csv('../Data/Output/predictions_final_append.csv')  #Predictions for all members

In [553]:
predictions_final_df = predictions_final_append.copy()

In [554]:
predictions_final_df.drop(('BookID') ,axis=1, inplace=True)

In [555]:
predictions_final_df.shape

(15000, 11)

In [556]:
predictions_final_df.to_csv('../Data/Output/predictions_final_df.csv')  #FINAL Predictions for all members